In [1]:
import numpy as np
import pandas as pd
import cvxpy as cp
import mosek
import matplotlib.pyplot as plt
import scipy.stats
import phi_divergence as phi
from time import process_time


In [63]:
def toymodel(Z_arr):
    x = cp.Variable(2, nonneg = True)
    constraints = [Z_arr @ x <= 1, x[0]<= 1+x[1], x[0] <= x[1]-1, x<=10]
    obj = cp.Maximize(cp.sum(x))
    prob = cp.Problem(obj,constraints)
    prob.solve(solver=cp.MOSEK)
    return(x.value, prob.value)

def lowbound(x,p,r, par, phi_div):
    q = cp.Variable(2, nonneg = True)
    constraints = [cp.sum(q) == 1]
    constraints = phi_div(p,q,r,par,constraints)
    obj = cp.Minimize(q[0])
    prob = cp.Problem(obj,constraints)
    prob.solve(solver=cp.MOSEK)
    return(prob.value)

In [101]:
par = 1
phi_div = phi.mod_chi2_cut
n = 1000
phi_dot = 1
r = phi_dot/(2*n)*scipy.stats.chi2.ppf(0.95, 1)

In [479]:
np.random.seed(1)
data = np.random.uniform(-1,1,size = (n,2))
Z_arr = np.array([data[0]])
beta = 0.85
delta = 1/n

In [461]:
[x, obj] = toymodel(Z_arr)
print('true prob', 1/2+1/(2*x[1]))
constr = data.dot(x)
p = np.array([len(constr[constr<=1])/n,len(constr[constr>1])/n])
lb = lowbound(x,p,r, par, phi_div)
print(obj)
print(lb)
gap = np.ceil(n*(beta-lb)).astype(int)
print(gap)
if gap > 0:
    vio_sort = np.sort(constr[constr>1])
    vio_value = vio_sort[gap-1]
    ind = np.where(constr == vio_value)[0][0]
    Z_arr = np.append(Z_arr, np.array([data[ind]]),axis = 0)

true prob 0.9076476882939759
1.4530986651369198
0.8741786303566359
-24


In [480]:
lb = -np.inf
while beta > lb:
    [x, obj] = toymodel(Z_arr)
    constr = data.dot(x)
    p = np.array([len(constr[constr<=1])/n,len(constr[constr>1])/n])
    lb = lowbound(x,p,r, par, phi_div)
    if len(constr[constr>1])!= 0:
        vio_min = np.min(constr[constr>1])
        ind = np.where(constr == vio_min)[0][0]
        Z_arr = np.append(Z_arr, np.array([data[ind]]),axis = 0)
print(obj)
print(lb)
print('true prob', 1/2+1/(2*x[1]))

1.63915710331428
0.850023573814368
true prob 0.87890885644671
